### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42303593,79085,86975,51530,0,0,1123,5,2025-01-16 22:00,2025-01-16 22:00,0,2,3904,5,1
34,42305274,28120,79085,3,0,0,1123,6,2025-01-17 19:20,2025-01-17 19:21,1,0,6673,4,1
35,42309239,79085,107614,51530,0,0,1124,0,2025-01-18 22:00,2025-01-18 22:00,1,0,2083,2,1
36,42084940,23755,79085,1349,9,69,1124,1,2025-01-19 21:58,2025-01-19 21:59,2,2,23066,2,1
37,42309240,79085,107614,51530,0,0,1124,2,2025-01-20 22:00,2025-01-20 22:00,0,5,1935,0,1
38,42311611,79085,84954,51530,0,0,1124,3,2025-01-21 22:00,2025-01-21 22:00,0,3,2559,3,1
39,42084956,122169,79085,1349,10,69,1124,4,2025-01-22 21:59,2025-01-22 21:59,1,1,7654,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,40508,34500,2,3,0,0,0,4-3-3a,8,9,10,42303593
1,86975,29451,35459,1,11,3,0,0,DEUTSCH,9,7,9,42303593
0,28120,36189,35767,3,5,1,1,0,Smrtka 3,5,6,7,42305274
1,79085,33768,34190,0,6,2,0,0,4-3-3aRU,6,7,6,42305274
0,79085,46163,34322,1,7,2,1,0,4-4-2a,9,9,10,42309239
1,107614,23787,35628,1,10,4,1,0,Her_wind_b,9,10,9,42309239
0,23755,28747,35104,2,9,0,0,0,efc,10,9,11,42084940
1,79085,41525,35168,0,3,1,0,0,4-4-2c,9,9,10,42084940
0,79085,46599,37606,2,3,5,0,0,4-4-2a,8,9,11,42309240
1,107614,23467,32460,0,13,3,1,0,Her_wind_b,12,10,10,42309240


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1518

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,10,30,10,0,0,45,6,30539045010010,1349
1,6295,10,24,8,0,2,57,21,24536057008009,1349
2,23755,10,20,6,2,2,50,16,20534050006011,1349
3,79866,10,20,6,2,2,24,10,20514024006005,1349
4,78671,10,18,6,0,4,38,15,18523038006007,1349
5,89569,10,16,5,1,4,25,21,16504025005002,1349
6,124139,10,15,5,0,5,30,19,15511030005008,1349
7,122169,10,13,4,1,5,50,34,13516050004004,1349
8,79085,10,13,3,4,3,20,11,13509020003006,1349
9,48541,10,3,1,0,9,9,72,3437009001001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()